In [39]:
pip install opencv-python face-recognition numpy python-doctr torch torchvision geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [40]:
# %%
import os
import tempfile
import time
import re
import tkinter as tk
from tkinter import filedialog, messagebox
from datetime import datetime
from threading import Thread

import cv2
import numpy as np
import face_recognition
from PIL import Image, ImageTk
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time as _time

# Cargar modelo OCR una sola vez (puede tardar)
print("Cargando modelo OCR (docTR)...")
OCR_MODEL = ocr_predictor(pretrained=True)
print("Modelo OCR listo.")

Cargando modelo OCR (docTR)...
Modelo OCR listo.


In [41]:
# %%
def cargar_encoding_dni(dni_path):
    img = face_recognition.load_image_file(dni_path)
    encodings = face_recognition.face_encodings(img)
    if encodings:
        return encodings[0]
    scale = 2
    img_large = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    encodings = face_recognition.face_encodings(img_large, num_jitters=1)
    if encodings:
        return encodings[0]
    face_locs = face_recognition.face_locations(img_large, number_of_times_to_upsample=2, model="hog")
    if not face_locs:
        raise ValueError("No se detectó ningún rostro en la imagen del DNI.")
    encodings = face_recognition.face_encodings(img_large, known_face_locations=face_locs, num_jitters=1)
    if not encodings:
        raise ValueError("No se detectó ningún rostro en la imagen del DNI.")
    return encodings[0]


def indice_rostro_principal(face_locations):
    areas = []
    for i, (top, right, bottom, left) in enumerate(face_locations):
        areas.append((i, max(0, right - left) * max(0, bottom - top)))
    return max(areas, key=lambda item: item[1])[0]


def ratio_nariz_cara(landmarks):
    chin = landmarks["chin"]
    nose_tip = landmarks["nose_tip"]
    left_x = chin[0][0]
    right_x = chin[-1][0]
    nose_x = int(np.mean([p[0] for p in nose_tip]))
    width = max(1, right_x - left_x)
    return (nose_x - left_x) / width


def detectar_dni_en_frame(frame_gray, dni_gray, orb, bf, min_matches=15):
    """Detecta si el DNI frontal está visible en el frame usando ORB feature matching."""
    kp_dni, des_dni = orb.detectAndCompute(dni_gray, None)
    kp_frame, des_frame = orb.detectAndCompute(frame_gray, None)
    if des_dni is None or des_frame is None:
        return False
    matches = bf.match(des_dni, des_frame)
    matches = sorted(matches, key=lambda x: x.distance)
    buenos = [m for m in matches if m.distance < 60]
    return len(buenos) >= min_matches


def redimensionar_para_mostrar(frame, max_w=1280, max_h=720):
    """Redimensiona el frame si supera el tamaño máximo, manteniendo proporción."""
    h, w = frame.shape[:2]
    if w <= max_w and h <= max_h:
        return frame
    escala = min(max_w / w, max_h / h)
    nuevo_w = int(w * escala)
    nuevo_h = int(h * escala)
    return cv2.resize(frame, (nuevo_w, nuevo_h), interpolation=cv2.INTER_AREA)


def verificar_en_vivo(dni_path, cam_id=0, tolerance=0.60, timeout_seg=120):
    dni_encoding = cargar_encoding_dni(dni_path)

    # Preparar imagen del DNI para feature matching (ORB)
    dni_img = cv2.imread(dni_path)
    if dni_img is None:
        raise IOError("No se puede leer la imagen del DNI.")
    dni_gray = cv2.cvtColor(dni_img, cv2.COLOR_BGR2GRAY)
    dni_gray = cv2.resize(dni_gray, (400, int(400 * dni_gray.shape[0] / dni_gray.shape[1])))
    orb = cv2.ORB_create(nfeatures=1000)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    cap = cv2.VideoCapture(cam_id, cv2.CAP_DSHOW)
    if not cap.isOpened():
        raise IOError("No se puede abrir la cámara.")

    # Estado de las fases
    giro_izq_ok = False
    giro_der_ok = False
    giros_completados = False
    dni_con_cara_ok = False
    distancias_validas = []
    inicio = time.time()

    try:
        for _ in range(3):
            cap.read()

        while (time.time() - inicio) < timeout_seg:
            ret, frame = cap.read()
            if not ret:
                continue
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # ============================================================
            # FASE 1: Prueba de vida - Girar la cabeza a ambos lados
            # ============================================================
            if not giros_completados:
                face_locations = face_recognition.face_locations(rgb, model="hog")
                if not face_locations:
                    cv2.putText(frame, "FASE 1: Gire la cabeza a ambos lados",
                                (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 200, 255), 2)
                    cv2.putText(frame, "No se detecta rostro",
                                (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    cv2.imshow("Verificacion en vivo", redimensionar_para_mostrar(frame))
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break
                    continue

                idx = indice_rostro_principal(face_locations)
                loc = [face_locations[idx]]
                lms = face_recognition.face_landmarks(rgb, face_locations=loc)
                if lms:
                    landmarks = lms[0]
                    if "chin" in landmarks and "nose_tip" in landmarks:
                        ratio = ratio_nariz_cara(landmarks)
                        if ratio < 0.42:
                            giro_izq_ok = True
                        if ratio > 0.58:
                            giro_der_ok = True

                giros_completados = giro_izq_ok and giro_der_ok

                cv2.putText(frame, "FASE 1: Gire la cabeza a ambos lados",
                            (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 200, 255), 2)
                cv2.putText(frame,
                            f"GiroIzq: {'OK' if giro_izq_ok else '...'} | GiroDer: {'OK' if giro_der_ok else '...'}",
                            (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.putText(frame, "Q para salir",
                            (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
                cv2.imshow("Verificacion en vivo", redimensionar_para_mostrar(frame))
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                continue

            # ============================================================
            # FASE 2: Sostener el DNI junto a la cara mirando a la cámara
            # ============================================================
            face_locations = face_recognition.face_locations(rgb, model="hog")
            dni_visible = detectar_dni_en_frame(frame_gray, dni_gray, orb, bf, min_matches=15)
            tiene_rostro = len(face_locations) > 0

            if tiene_rostro and dni_visible:
                # Ambos detectados: comparar rostro con el del DNI
                idx = indice_rostro_principal(face_locations)
                loc = [face_locations[idx]]
                encs = face_recognition.face_encodings(rgb, known_face_locations=loc)
                if encs:
                    distancia = face_recognition.face_distance([dni_encoding], encs[0])[0]
                    distancias_validas.append(float(distancia))

            # Indicadores en pantalla
            color_rostro = (0, 255, 0) if tiene_rostro else (0, 0, 255)
            color_dni = (0, 255, 0) if dni_visible else (0, 0, 255)

            cv2.putText(frame, "FASE 2: Muestre el DNI junto a su cara",
                        (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 200, 255), 2)
            cv2.putText(frame,
                        f"Rostro: {'OK' if tiene_rostro else 'NO'}", (20, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_rostro, 2)
            cv2.putText(frame,
                        f"DNI: {'DETECTADO' if dni_visible else 'NO DETECTADO'}", (20, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_dni, 2)

            if distancias_validas:
                cv2.putText(frame, f"Muestras: {len(distancias_validas)}/6",
                            (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            cv2.putText(frame, "Sostenga el DNI al lado de su cara mirando a la camara (Q salir)",
                        (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200, 200, 200), 1)

            cv2.imshow("Verificacion en vivo", redimensionar_para_mostrar(frame))
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            if len(distancias_validas) >= 6:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

    if not giros_completados:
        raise ValueError("Error: prueba de vida no superada (giros de cabeza).")
    if not distancias_validas:
        raise ValueError("Error: no se detectó su rostro junto al DNI.")

    distancia_final = float(np.median(distancias_validas))
    coincide = distancia_final <= tolerance
    return coincide, distancia_final

In [42]:
# %%
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time as _time

_geolocator = Nominatim(user_agent="dni_verificacion_hackaton", timeout=5)


def verificar_ubicacion_existe(texto, es_domicilio=False):
    """
    Verifica que una ubicación (ciudad o dirección) existe en España usando Nominatim.
    Retorna (existe: bool, ubicacion_encontrada: str).
    """
    if not texto or len(texto.strip()) < 3:
        return False, "Texto muy corto"
    
    consulta = f"{texto.strip()}, España"
    try:
        _time.sleep(1.1)  # Nominatim requiere max 1 petición/segundo
        resultado = _geolocator.geocode(consulta, country_codes="es", language="es")
        if resultado:
            return True, resultado.address
        # Intentar solo con la parte principal (quitar números de portal, piso, etc.)
        if es_domicilio:
            # Extraer solo el nombre de la calle/ciudad
            partes = re.split(r'\d+', texto.strip())
            parte_limpia = partes[0].strip() if partes else texto.strip()
            if parte_limpia and len(parte_limpia) >= 3 and parte_limpia != texto.strip():
                _time.sleep(1.1)
                resultado = _geolocator.geocode(f"{parte_limpia}, España", country_codes="es", language="es")
                if resultado:
                    return True, resultado.address
        return False, "No encontrada"
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        return False, f"Error de servicio: {e}"
    except Exception as e:
        return False, f"Error: {e}"


def extraer_domicilio_y_nacimiento(lineas_reverso):
    """
    Intenta extraer el domicilio y lugar de nacimiento del texto OCR del reverso del DNI.
    Soporta etiquetas en castellano, catalán/valenciano, euskera y gallego.
    Retorna (domicilio, lugar_nacimiento).
    """
    # Patrones de etiquetas en los 4 idiomas cooficiales (tolerante a errores OCR)
    PAT_DOM = (r'DOM[I1][CGK][I1]?L?[I1]?[OUQ0]?|HELBIDEA')
    PAT_NAC = (r'LUGAR\s*(DE\s*)?NACIMIENTO'
               r'|LLOC\s*(DE\s*)?NAIXEMENT'
               r'|JAIO(TZE|TZA)\s*(LEKUA|TOKIA)'
               r'|LUGAR\s*(DE\s*)?NACEMENTO'
               r'|L\.\s*NACIMIENTO'
               r'|L\.\s*NAIXEMENT')
    # Para limpiar la etiqueta del texto extraído
    PAT_NAC_CLEAN = (r'LUGAR\s*(DE\s*)?NACIMIENTO|LLOC\s*(DE\s*)?NAIXEMENT'
                     r'|JAIO(TZE|TZA)\s*(LEKUA|TOKIA)|LUGAR\s*(DE\s*)?NACEMENTO'
                     r'|L\.\s*NACIMIENTO|L\.\s*NAIXEMENT|/?\s*NACIM\w*')
    # Etiquetas que indican otro campo (para no concatenar líneas de más)
    PAT_OTRO_CAMPO = (r'LUGAR|NACIM|LLOC|NAIX|JAIO|MRZ|IDESP|<|HELBIDE|DOM[I1][CG]')

    domicilio = ""
    lugar_nacimiento = ""

    for i, linea in enumerate(lineas_reverso):
        linea_upper = linea.upper().strip()

        # --- Domicilio ---
        if not domicilio and re.search(PAT_DOM, linea_upper):
            # Quitar toda la etiqueta (posiblemente dos idiomas separados por /)
            resto = re.sub(r'^.*?(' + PAT_DOM + r')[:\s/]*', '', linea_upper, flags=re.IGNORECASE)
            # Si queda otra etiqueta del otro idioma, quitarla también
            resto = re.sub(PAT_DOM, '', resto, flags=re.IGNORECASE).strip(' /:')
            if resto and len(resto) > 3:
                domicilio = resto
            elif i + 1 < len(lineas_reverso):
                domicilio = lineas_reverso[i + 1].strip()
            # Concatenar siguiente línea si no es otro campo
            if domicilio and i + 2 < len(lineas_reverso):
                siguiente = lineas_reverso[i + 2].strip()
                if siguiente and not re.search(PAT_OTRO_CAMPO, siguiente.upper()):
                    domicilio = f"{domicilio} {siguiente}"

        # --- Lugar de nacimiento ---
        if not lugar_nacimiento and re.search(PAT_NAC, linea_upper):
            # Quitar toda la etiqueta (puede haber varias en distintos idiomas separadas por /)
            resto = re.sub(r'^[^A-Za-z]*', '', linea_upper)  # quitar basura al inicio
            resto = re.sub(PAT_NAC_CLEAN, '', resto, flags=re.IGNORECASE).strip(' /:')
            # Quitar barras sueltas y etiquetas residuales
            resto = re.sub(r'^[/\s]+', '', resto).strip()
            if resto and len(resto) > 2:
                lugar_nacimiento = resto
            elif i + 1 < len(lineas_reverso):
                sig = lineas_reverso[i + 1].strip()
                # Verificar que la siguiente línea no sea otra etiqueta
                if sig and not re.search(PAT_NAC + '|' + PAT_DOM + '|IDESP|<', sig.upper()):
                    lugar_nacimiento = sig

    return domicilio.strip(), lugar_nacimiento.strip()


def extraer_texto_ocr(imagen_path, confidence=0.5):
    """Ejecuta OCR sobre una imagen y devuelve lista de líneas de texto."""
    try:
        doc = DocumentFile.from_images(imagen_path)
        result = OCR_MODEL(doc)
        lineas = []
        for page in result.pages:
            for block in page.blocks:
                for line in block.lines:
                    palabras = [word.value for word in line.words if word.confidence > confidence]
                    if palabras:
                        lineas.append(" ".join(palabras))
        return lineas
    except Exception as e:
        raise RuntimeError(f"Error en OCR de {imagen_path}: {str(e)}")


def validar_datos_dni(anverso_path, reverso_path):
    """
    Valida los datos del DNI comparando anverso y MRZ del reverso.
    También verifica domicilio y lugar de nacimiento contra Nominatim.
    Retorna (exito, mensaje, detalles) donde detalles es un dict con los campos.
    """
    hoy = datetime.now()
    try:
        # OCR anverso y reverso
        f_lines = extraer_texto_ocr(anverso_path, 0.5)
        b_lines = extraer_texto_ocr(reverso_path, 0.4)
        f_all = " ".join(f_lines).upper()

        # DEBUG: ver líneas OCR del reverso
        print("[DEBUG] === Líneas OCR reverso ===")
        for idx_l, linea in enumerate(b_lines):
            print(f"  [{idx_l}] {linea}")
        print("[DEBUG] ===========================")

        # Buscar líneas candidatas a MRZ en el reverso
        m_c = [l.replace(" ", "") for l in b_lines if len(l.replace(" ", "")) >= 28 and ("<" in l or "ESP" in l)]
        if len(m_c) < 3:
            return False, "No se detectaron las 3 líneas del MRZ en el reverso.", {}

        # Identificar las tres líneas del MRZ (asumiendo orden)
        l1 = next((l for l in m_c if "ESP" in l[:10]), m_c[0])
        idx = m_c.index(l1)
        l2 = m_c[idx+1] if len(m_c) > idx+1 else m_c[1]
        l3 = m_c[idx+2] if len(m_c) > idx+2 else m_c[-1]

        # Parsear campos del MRZ (estructura típica DNI español)
        m_data = {
            "id": l1[5:14].replace("<", ""),                     # IDESP
            "dni": l1[15:23].replace("<", ""),                   # Número DNI
            "s": l2[7],                                           # Sexo
            "b": f"{l2[4:6]} {l2[2:4]} 20{l2[0:2]}",            # Fecha nacimiento
            "e": f"{l2[12:14]} {l2[10:12]} 20{l2[8:10]}",        # Fecha caducidad
            "n": l3.replace("<", " ").strip()                     # Nombre
        }

        # Extraer fechas del anverso
        dates = re.findall(r'\d{2} \d{2} \d{4}', f_all)
        iss = next((d for d in dates if d != m_data["b"] and d != m_data["e"]), "N/A")

        # Función para validar fecha respecto a hoy
        def fecha_valida(fecha_str, debe_ser_pasada):
            try:
                d = datetime.strptime(fecha_str.replace(" ", ""), "%d%m%Y")
                return (d < hoy) if debe_ser_pasada else (d > hoy)
            except:
                return False

        # Validar fechas
        emision_ok = fecha_valida(iss, debe_ser_pasada=True) if iss != "N/A" else False
        caducidad_ok = fecha_valida(m_data["e"], debe_ser_pasada=False)

        # Comparar campos con el anverso
        dni_f = (re.search(r'\d{8}[A-Z]', f_all) or re.search(r'\d{8}', f_all)).group(0) if re.search(r'\d{8}', f_all) else "N/A"
        id_f = re.search(r'[A-Z]{3}\d{6}', f_all).group(0) if re.search(r'[A-Z]{3}\d{6}', f_all) else "N/A"

        dni_ok = (dni_f[:8] == m_data["dni"]) if dni_f != "N/A" else False
        id_ok = (id_f == m_data["id"]) if id_f != "N/A" else False
        sexo_ok = m_data["s"] in f_all
        palabras_nombre = [p for p in m_data["n"].replace("K", " ").split() if len(p) > 3]
        nombre_ok = any(p in f_all for p in palabras_nombre) if palabras_nombre else False

        # --- Validar domicilio y lugar de nacimiento ---
        domicilio, lugar_nacimiento = extraer_domicilio_y_nacimiento(b_lines)
        print(f"[DEBUG] Domicilio extraído: '{domicilio}'")
        print(f"[DEBUG] Lugar de nacimiento extraído: '{lugar_nacimiento}'")

        domicilio_ok = False
        domicilio_info = "No detectado"
        if domicilio:
            domicilio_ok, domicilio_info = verificar_ubicacion_existe(domicilio, es_domicilio=True)
            print(f"[DEBUG] Nominatim domicilio -> ok={domicilio_ok}, info='{domicilio_info}'")

        nacimiento_lugar_ok = False
        nacimiento_lugar_info = "No detectado"
        if lugar_nacimiento:
            nacimiento_lugar_ok, nacimiento_lugar_info = verificar_ubicacion_existe(lugar_nacimiento, es_domicilio=False)
            print(f"[DEBUG] Nominatim lugar nacimiento -> ok={nacimiento_lugar_ok}, info='{nacimiento_lugar_info}'")

        # Coherencia global (domicilio y nacimiento son informativos, no bloquean)
        datos_base_ok = dni_ok and id_ok and sexo_ok and nombre_ok and emision_ok and caducidad_ok
        datos_ok = datos_base_ok  # Las ubicaciones se muestran pero no bloquean

        detalles = {
            "dni": (m_data["dni"], dni_f, dni_ok),
            "id_esp": (m_data["id"], id_f, id_ok),
            "sexo": (m_data["s"], "detectado" if sexo_ok else "no", sexo_ok),
            "nacimiento": (m_data["b"], iss if iss in dates else "N/A", emision_ok),
            "caducidad": (m_data["e"], "N/A", caducidad_ok),
            "nombre": (m_data["n"], "detectado" if nombre_ok else "no", nombre_ok),
            "emision": (iss, "N/A", emision_ok),
            "domicilio": (domicilio if domicilio else "No detectado", domicilio_info, domicilio_ok),
            "lugar_nacimiento": (lugar_nacimiento if lugar_nacimiento else "No detectado", nacimiento_lugar_info, nacimiento_lugar_ok),
        }

        if datos_ok:
            return True, "Datos del DNI válidos y vigentes.", detalles
        else:
            return False, "Algunos datos del DNI no coinciden o están fuera de vigencia.", detalles

    except Exception as e:
        return False, f"Error durante la validación: {str(e)}", {}

In [43]:

# %%
from PIL import Image, ImageTk   # importar aquí por si la celda de imports no se re-ejecutó

class App:
    PREVIEW_MAX_W = 400
    PREVIEW_MAX_H = 260

    def __init__(self, root):
        self.root = root
        self.root.title("Verificación de Identidad Completa")
        self.root.geometry("680x900")
        self.root.resizable(False, False)

        self.anverso_path = None
        self.reverso_path = None

        # Referencias para evitar que el garbage collector elimine las imágenes
        self._img_anverso = None
        self._img_reverso = None

        # Variables de estado
        self.estado_anverso = tk.StringVar(value="[PENDIENTE] No cargado")
        self.estado_reverso = tk.StringVar(value="[PENDIENTE] No cargado")

        self.create_widgets()

    def create_widgets(self):
        # Título
        tk.Label(self.root, text="Verificación de Identidad", font=("Arial", 16, "bold")).pack(pady=10)

        # --- Frame para anverso ---
        frame_anverso = tk.LabelFrame(self.root, text="Anverso del DNI", padx=10, pady=5)
        frame_anverso.pack(fill="x", padx=20, pady=5)

        row_btns_anv = tk.Frame(frame_anverso)
        row_btns_anv.pack(fill="x")
        tk.Button(row_btns_anv, text="Cargar imagen", command=lambda: self.cargar_imagen("anverso"), width=15).pack(side="left", padx=5)
        tk.Button(row_btns_anv, text="Tomar foto", command=lambda: self.tomar_foto("anverso"), width=15).pack(side="left", padx=5)
        tk.Label(row_btns_anv, textvariable=self.estado_anverso, fg="blue").pack(side="left", padx=10)

        self.preview_anverso = tk.Label(frame_anverso, text="Sin previsualización", fg="gray",
                                         relief="sunken", width=50, height=16)
        self.preview_anverso.pack(pady=5)

        # --- Frame para reverso ---
        frame_reverso = tk.LabelFrame(self.root, text="Reverso del DNI", padx=10, pady=5)
        frame_reverso.pack(fill="x", padx=20, pady=5)

        row_btns_rev = tk.Frame(frame_reverso)
        row_btns_rev.pack(fill="x")
        tk.Button(row_btns_rev, text="Cargar imagen", command=lambda: self.cargar_imagen("reverso"), width=15).pack(side="left", padx=5)
        tk.Button(row_btns_rev, text="Tomar foto", command=lambda: self.tomar_foto("reverso"), width=15).pack(side="left", padx=5)
        tk.Label(row_btns_rev, textvariable=self.estado_reverso, fg="blue").pack(side="left", padx=10)

        self.preview_reverso = tk.Label(frame_reverso, text="Sin previsualización", fg="gray",
                                         relief="sunken", width=50, height=16)
        self.preview_reverso.pack(pady=5)

        # Botón de verificación
        self.btn_verificar = tk.Button(self.root, text="Iniciar verificación completa",
                                       command=self.iniciar_verificacion, state=tk.DISABLED,
                                       bg="lightblue", font=("Arial", 12), padx=10, pady=5)
        self.btn_verificar.pack(pady=10)

        # Barra de estado
        self.status_var = tk.StringVar(value="Listo. Cargue ambas caras del DNI.")
        self.status_label = tk.Label(self.root, textvariable=self.status_var, bg="#e5e7eb", font=("Arial", 9))
        self.status_label.pack(fill="x", side="bottom")

    def _mostrar_preview(self, ruta, lado):
        """Carga la imagen, la redimensiona y la muestra en el label de previsualización."""
        try:
            img_cv = cv2.imread(ruta)
            if img_cv is None:
                print(f"Preview: no se pudo leer {ruta}")
                return
            img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            # Redimensionar manteniendo proporción al tamaño máximo
            img_pil.thumbnail((self.PREVIEW_MAX_W, self.PREVIEW_MAX_H), Image.LANCZOS)
            tk_img = ImageTk.PhotoImage(img_pil)

            if lado == "anverso":
                self._img_anverso = tk_img
                self.preview_anverso.config(image=tk_img, text="", width=img_pil.width, height=img_pil.height)
            else:
                self._img_reverso = tk_img
                self.preview_reverso.config(image=tk_img, text="", width=img_pil.width, height=img_pil.height)
        except Exception as e:
            print(f"Error en previsualización: {e}")

    def cargar_imagen(self, lado):
        file_path = filedialog.askopenfilename(
            title=f"Seleccionar imagen del {lado}",
            filetypes=[("Archivos de imagen", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        if file_path:
            self._asignar_imagen(lado, file_path)

    def tomar_foto(self, lado):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "No se pudo abrir la cámara.")
            return

        nombre_ventana = f"Capturar {lado} - ESPACIO para tomar, ESC cancelar"
        img_capturada = None

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            cv2.putText(frame, "ESPACIO para capturar, ESC para cancelar", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.imshow(nombre_ventana, redimensionar_para_mostrar(frame))
            key = cv2.waitKey(1) & 0xFF
            if key == 27:
                break
            elif key == 32:
                img_capturada = frame.copy()
                break

        cap.release()
        cv2.destroyAllWindows()

        if img_capturada is not None:
            temp_dir = tempfile.gettempdir()
            temp_path = os.path.join(temp_dir, f"dni_{lado}_{int(time.time())}.jpg")
            cv2.imwrite(temp_path, img_capturada)
            self._asignar_imagen(lado, temp_path)

    def _asignar_imagen(self, lado, ruta):
        if lado == "anverso":
            self.anverso_path = ruta
            self.estado_anverso.set(f"[OK] {os.path.basename(ruta)}")
        else:
            self.reverso_path = ruta
            self.estado_reverso.set(f"[OK] {os.path.basename(ruta)}")

        # Mostrar previsualización
        self._mostrar_preview(ruta, lado)

        # Habilitar botón si ambas cargas están listas
        if self.anverso_path and self.reverso_path:
            self.btn_verificar.config(state=tk.NORMAL)
            self.status_var.set("Ambas caras cargadas. Pulse 'Iniciar verificación'.")
        else:
            self.btn_verificar.config(state=tk.DISABLED)

    def iniciar_verificacion(self):
        if not self.anverso_path or not self.reverso_path:
            messagebox.showwarning("Advertencia", "Debe cargar ambas caras del DNI.")
            return

        # Deshabilitar botón durante el proceso
        self.btn_verificar.config(state=tk.DISABLED)
        self.status_var.set("Validando datos del DNI... (puede tardar unos segundos)")

        # Ejecutar validación en un hilo para no bloquear la UI
        Thread(target=self._proceso_validacion_y_biometria, daemon=True).start()

    def _proceso_validacion_y_biometria(self):
        try:
            # Paso 1: Validar datos del DNI (MRZ)
            exito, mensaje, detalles = validar_datos_dni(self.anverso_path, self.reverso_path)

            if not exito:
                self.root.after(0, lambda msg=mensaje: self._mostrar_error_validacion(msg))
                return

            # Si los datos son válidos, mostrar resumen y preguntar si continuar
            resumen = self._formatear_resumen(detalles)
            continuar = messagebox.askyesno("Datos válidos",
                                            f"Los datos del DNI son correctos y vigentes.\n\n{resumen}\n\n¿Desea continuar con la verificación biométrica?")
            if not continuar:
                self.root.after(0, lambda: self._finalizar_proceso(False, "Verificación cancelada por el usuario."))
                return

            # Paso 2: Verificación biométrica en vivo
            self.root.after(0, lambda: self.status_var.set("Iniciando verificación en vivo... (mire a la cámara)"))
            coincide, distancia = verificar_en_vivo(
                dni_path=self.anverso_path,
                cam_id=0,
                tolerance=0.60,
                timeout_seg=120
            )

            if coincide:
                msg_ok = f"Verificación exitosa.\nCoincidencia biométrica con distancia: {distancia:.4f}"
                self.root.after(0, lambda m=msg_ok: self._finalizar_proceso(True, m))
            else:
                msg_fail = f"La persona no coincide con el DNI (distancia: {distancia:.4f})."
                self.root.after(0, lambda m=msg_fail: self._finalizar_proceso(False, m))

        except Exception as e:
            error_msg = f"Error inesperado: {str(e)}"
            self.root.after(0, lambda msg=error_msg: self._mostrar_error_validacion(msg))

    def _formatear_resumen(self, detalles):
        lineas = []
        for campo, (valor_mrz, valor_front, ok) in detalles.items():
            estado = "[OK]" if ok else "[ERROR]"
            if campo == "emision":
                lineas.append(f"Emisión: {valor_mrz} {estado}")
            elif campo == "caducidad":
                lineas.append(f"Caducidad: {valor_mrz} {estado}")
            elif campo == "nombre":
                lineas.append(f"Nombre: {valor_mrz[:30]}... {estado}")
            elif campo == "dni":
                lineas.append(f"DNI: {valor_mrz} (frontal: {valor_front}) {estado}")
            elif campo == "id_esp":
                lineas.append(f"IDESP: {valor_mrz} (frontal: {valor_front}) {estado}")
            elif campo == "sexo":
                lineas.append(f"Sexo: {valor_mrz} {estado}")
            elif campo == "nacimiento":
                lineas.append(f"Nacimiento: {valor_mrz} {estado}")
            elif campo == "domicilio":
                lineas.append(f"Domicilio: {valor_mrz[:40]} {estado}")
                if ok:
                    lineas.append(f"  → {valor_front[:60]}")
            elif campo == "lugar_nacimiento":
                lineas.append(f"Lugar nacimiento: {valor_mrz[:40]} {estado}")
                if ok:
                    lineas.append(f"  → {valor_front[:60]}")
        return "\n".join(lineas)

    def _mostrar_error_validacion(self, mensaje):
        messagebox.showerror("Error de validación", mensaje)
        self.status_var.set("Validación fallida. Revise las imágenes.")
        self.btn_verificar.config(state=tk.NORMAL)

    def _finalizar_proceso(self, exito, mensaje):
        if exito:
            messagebox.showinfo("Éxito", mensaje)
            self.status_var.set("Verificación completada con éxito.")
        else:
            messagebox.showerror("Error", mensaje)
            self.status_var.set("Verificación fallida.")
        self.btn_verificar.config(state=tk.NORMAL)

In [45]:
# %%
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()

[DEBUG] === Líneas OCR reverso ===
  [0] DOMICILIO/DOMIGIU
  [1] AVDA. NACIONES 12 P04 B
  [2] PLAYA SAN JUAN
  [3] ALICANTE
  [4] ALICANTE
  [5] LUGAR DE NACIMIENTO/LLOCDE NAIXEMENT
  [6] SANT JOAN D'ALACANT
  [7] ALICANTE
  [8] I
  [9] HIJO/A DE/FILL/AI DE
  [10] I
  [11] JUAN ANTONIO / SUETLANA VASILIEVNA
  [12] IDESPCEE1724514486749142<<<<<<
  [13] 0312129M2801175ESP<<<<<<<<<<<
  [14] CORTES<NETLENNIIK<SERGIC<<<<<<
[DEBUG] ===========================
[DEBUG] Domicilio extraído: 'AVDA. NACIONES 12 P04 B PLAYA SAN JUAN'
[DEBUG] Lugar de nacimiento extraído: 'SANT JOAN D'ALACANT'
[DEBUG] Nominatim domicilio -> ok=True, info='Avenida de las Naciones, Alicante, L'Alacantí, Alicante, Comunidad Valenciana, 03540, España'
[DEBUG] Nominatim lugar nacimiento -> ok=True, info='San Juan de Alicante, L'Alacantí, Alicante, Comunidad Valenciana, 03550, España'
[DEBUG] === Líneas OCR reverso ===
  [0] DOMICILIO/DOMIGIU
  [1] AVDA. NACIONES 12 P04 B
  [2] PLAYA SAN JUAN
  [3] ALICANTE
  [4] ALICANT